In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI
private_key = "" # Your own secret key
organization_id = "" # SKI-ML Organization ID
project_id = "" # Samsung AI_Expert project ID

client = OpenAI(api_key=private_key,
                organization=organization_id,
               )

In [ ]:
from openai import OpenAI

def chatgpt(prompt: str) -> str:
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
    ]
  )
  return completion.choices[0].message.content.strip()

In [ ]:
### Test

chatgpt("hello")

# Zero-shot prompting
Large language models (LLMs) today, such as GPT-3.5 Turbo, GPT-4, and Claude 3, are tuned to follow instructions and are trained on large amounts of data. Large-scale training makes these models capable of performing some tasks in a "zero-shot" manner. Zero-shot prompting means that the prompt used to interact with the model won't contain examples or demonstrations. The zero-shot prompt directly instructs the model to perform a task without any additional examples to steer it.

In [ ]:
ZERO_SHOT="""Classify the text into neutral, negative or positive.

Text: I think the vacation is okay.
Sentiment:"""
chatgpt(ZERO_SHOT)

Write your own zero shot prompt (in English or Korean)

In [ ]:
YOUR_ZERO_SHOT_PROMPT="""
"""
chatgpt(YOUR_ZERO_SHOT_PROMPT)

# Few-shot prompting
While large-language models demonstrate remarkable zero-shot capabilities, they still fall short on more complex tasks when using the zero-shot setting. Few-shot prompting can be used as a technique to enable in-context learning where we provide demonstrations in the prompt to steer the model to better performance. The demonstrations serve as conditioning for subsequent examples where we would like the model to generate a response.

In [ ]:
FEW_SHOT="""
Input: This is awesome!
Output: Positive

Input: This is bad!
Output: Negative

Input: Wow that movie was rad!
Output: Positive

Input: What a horrible show!
Output: """
chatgpt(FEW_SHOT)

Write your own few shot prompt (in English)

In [ ]:
ZERO_SHOT="""Find the next number in the sequence. Each number is doubled to get the next number.
Input: 2, 4, 8, 16
Output:
"""
print(chatgpt(ZERO_SHOT))

In [ ]:
YOUR_FEW_SHOT_PROMPT="""Find the next number in the sequence. Each number is doubled to get the next number.
"""
print(chatgpt(YOUR_FEW_SHOT_PROMPT))

# Chain-of-Thought Prompting
Introduced in Wei et al. (2022), chain-of-thought (CoT) prompting enables complex reasoning capabilities through intermediate reasoning steps. You can combine it with few-shot prompting to get better results on more complex tasks that require reasoning before responding.

In [ ]:
COT="""The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: Adding all the odd numbers (17, 19) gives 36. The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: Adding all the odd numbers (11, 13) gives 24. The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: Adding all the odd numbers (17, 9, 13) gives 39. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:"""
chatgpt(COT)

## Zero-shot CoT

In [ ]:
NORMAL_PROMPT=("I went to the market and bought 10 apples. "
               "I gave 2 apples to the neighbor and 2 to the repairman. "
               "I then went and bought 5 more apples and ate 1. "
               "How many apples did I remain with?")
chatgpt(NORMAL_PROMPT)

In [ ]:
ZERO_SHOT_COT=("I went to the market and bought 10 apples. "
               "I gave 2 apples to the neighbor and 2 to the repairman. "
               "I then went and bought 5 more apples and ate 1. "
               "How many apples did I remain with? "
               "Let's think step by step.")
chatgpt(ZERO_SHOT_COT)

# Prompt Chaining
To improve the reliability and performance of LLMs, one of the important prompt engineering techniques is to break tasks into its subtasks. Once those subtasks have been identified, the LLM is prompted with a subtask and then its response is used as input to another prompt. This is what's referred to as prompt chaining, where a task is split into subtasks with the idea to create a chain of prompt operations.

In [ ]:
# Define the first prompt
prompt_1 = "What are the key ingredients of a successful marketing campaign?"

# Get the response from the first prompt
response_1 = chatgpt(prompt_1)

# Define the second prompt, using the response from the first prompt as input
prompt_2 = f"Based on the key ingredients you mentioned, provide specific examples of how a business can implement them in their marketing campaign. Use the following format: \n\nIngredient: \nExample:\n\n{response_1}"

# Get the response from the second prompt
response_2 = chatgpt(prompt_2)

# Print the responses from both prompts
print(f"Response to prompt 1: {response_1}")
print(f"\nResponse to prompt 2: {response_2}")

Write your own chaining prompt (in English or Korean)

In [ ]:
prompt_1 = ""

response_1 = chatgpt(prompt_1)

prompt_2 = f"{response_1}"

response_2 = chatgpt(prompt_2)

print(f"Response to prompt 1: {response_1}")
print(f"\nResponse to prompt 2: {response_2}")

# Directional Stimulus Prompting
Li et al., (2023) proposes a new prompting technique to better guide the LLM in generating the desired summary.

A tuneable policy LM is trained to generate the stimulus/hint. Seeing more use of RL to optimize LLMs.

The figure below shows how Directional Stimulus Prompting compares with standard prompting. The policy LM can be small and optimized to generate the hints that guide a black-box frozen LLM.

![image](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fdsp.27a0005f.jpeg&w=3840&q=75)

In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

def calculate_rouge1(system_summary, reference_summary):
    # Create a ROUGE scorer for ROUGE-1
    scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    # Calculate the score
    scores = scorer.score(reference_summary, system_summary)
    return scores['rouge1']

In [ ]:
input_text = """Article: (CNN) For the first time in eight years, a TV legend returned to doing what he does best. Contestants told to "come on down!" on the April 1 edition of "The Price Is Right" encountered not host Drew Carey but another familiar face in charge of the proceedings. Instead, there was Bob Barker, who hosted the TV game show for 35 years before stepping down in 2007. Looking spry at 91, Barker handled the first price-guessing game of the show, the classic "Lucky Seven," before turning hosting duties over to Carey, who finished up. Despite being away from the show for most of the past eight years, Barker didn't seem to miss a beat."""
reference_text = """Bob Barker returned to host "The Price Is Right" on Wednesday. Barker, 91, had retired as host in 2007."""
NORMAL_PROMPT = f"""{input_text}\n
Q: Summarize the above article briefly in 2-3 sentences.

Output:
"""
DSP_PROMPT = f"""{input_text}\n
Q: Summarize the above article briefly in 2-3 sentences based on the hint.
Hint: Bob Barker; TV; April 1; "The Price Is Right"; 2007; 91.

Output:
"""
normal_summary = chatgpt(NORMAL_PROMPT)
dsp_summary = chatgpt(DSP_PROMPT)
normal_rouge1_score = calculate_rouge1(normal_summary, reference_text)
dsp_rouge1_score = calculate_rouge1(dsp_summary, reference_text)
print("ROUGE-1 Score (Normal Prompt):", normal_rouge1_score)
print("ROUGE-1 Score (DSP Prompt):", dsp_rouge1_score)


# ReACT Prompting
ReAct is inspired by the synergies between "acting" and "reasoning" which allow humans to learn new tasks and make decisions or reasoning.

Chain-of-thought (CoT) prompting has shown the capabilities of LLMs to carry out reasoning traces to generate answers to questions involving arithmetic and commonsense reasoning, among other tasks (Wei et al., 2022). But its lack of access to the external world or inability to update its knowledge can lead to issues like fact hallucination and error propagation.

ReAct is a general paradigm that combines reasoning and acting with LLMs. ReAct prompts LLMs to generate verbal reasoning traces and actions for a task. This allows the system to perform dynamic reasoning to create, maintain, and adjust plans for acting while also enabling interaction to external environments (e.g., Wikipedia) to incorporate additional information into the reasoning. The figure below shows an example of ReAct and the different steps involved to perform question answering.

In [ ]:
!pip install -U langchain-community langchain-core
!pip install -U google-search-results
!pip install -U langchain-openai
!pip install -U wikipedia

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import load_tools
from langchain.agents import initialize_agent, Tool

In [ ]:
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents import AgentType
from langchain import Wikipedia
docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", api_key=private_key, organization=organization_id)
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True, handle_parsing_errors=True)

react.run("Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?")

In [ ]:
## Print template used by LangChain
print(react.agent.llm_chain.prompt.template)